In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet


## Original Dataset

[The original dataset](https://www.kaggle.com/competitions/amex-default-prediction/overview) contains 3 relevant files: test_data.csv, train_data.csv, train_labels.csv. Since the dataset is a competition dataset only the training data is labelled.

For the original dataset the following description is given:

>The target binary variable is calculated by observing 18 months performance window after the latest credit card statement, and if the customer does not pay due amount in 120 days after their latest statement date it is considered a default event.

>The dataset contains aggregated profile features for each customer at each statement date. Features are anonymized and normalized, and fall into the following general categories:

>    * D_* = Delinquency variables
>    * S_* = Spend variables
>    * P_* = Payment variables
>    * B_* = Balance variables
>    * R_* = Risk variables

>with the following features being categorical:

>`['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']`

>Your task is to predict, for each customer_ID, the probability of a future payment default (target = 1).

Data analysis of the dataset which has been extremely helpful during the feature engineering process was performed by user @AmbrosM [here](https://www.kaggle.com/code/ambrosm/amex-eda-which-makes-sense#The-data)

## Our Dataset

For our use we'll use only the training data as the dataset and perform our own train-test split later.

List of modifications made:
- dataset converted from .csv format to parquet format
- artifical noise in original data removed
- float to int type conversions done

[Modified dataset](https://www.kaggle.com/datasets/raddar/amex-data-integer-dtypes-parquet-format/data)

## Preprocessing and Feature Engineering

In [2]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.5 MB/s eta 0:00:00


In [3]:
# loading in data
def read_file(path = ''):
    df = pd.read_parquet(path, engine='fastparquet')
    print('shape of data:', df.shape)
    return df

data = read_file(path='/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet')

shape of data: (5531451, 190)


In [4]:
data.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0


In [5]:
data.info(max_cols=200, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Data columns (total 190 columns):
 #    Column       Non-Null Count    Dtype  
---   ------       --------------    -----  
 0    customer_ID  5531451 non-null  object 
 1    S_2          5531451 non-null  object 
 2    P_2          5485466 non-null  float32
 3    D_39         5531451 non-null  int16  
 4    B_1          5531451 non-null  float32
 5    B_2          5529435 non-null  float32
 6    R_1          5531451 non-null  float32
 7    S_3          4510907 non-null  float32
 8    D_41         5529435 non-null  float32
 9    B_3          5529435 non-null  float32
 10   D_42         791314 non-null   float32
 11   D_43         3873055 non-null  float32
 12   D_44         5531451 non-null  int8   
 13   B_4          5531451 non-null  int16  
 14   D_45         5529434 non-null  float32
 15   B_5          5531451 non-null  float32
 16   R_2          5531451 non-null  int8   
 17   D_46         4319752 non-

In [6]:
# loading in targets
targets = pd.read_csv('../input/amex-default-prediction/train_labels.csv')

In [7]:
targets = targets.set_index('customer_ID')
targets.head()

,target
customer_ID,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0


The dataset is set up such that each row is a statement for a certain user, but the labels are set up such that each customer has a binary label. 

**We need to convert the dataset from individual statements to an aggregate of statements for each user.**

To do this we'll take the following aggregates for each user:

Numerical values:

* mean 
* standard deviation
* minimum value
* maximum value 
* last/most recent value

Categorical values:

* the count of how many times it has showed up 
* the last/most recent value it had 
* the number of times it has changed



In [8]:
def feature_engineer(df):
    features = [c for c in list(df.columns) if c not in ['customer_ID','S_2']] # all features except id and date
    cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
    num_features = [col for col in features if col not in cat_features]
    
    # calculate aggregates for numerical features
    test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    
    # calculate aggregates fot categorical features
    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

    df = pd.concat([test_num_agg, test_cat_agg], axis=1)
    del test_num_agg, test_cat_agg
    print('shape after engineering', df.shape )
    
    return df

data = feature_engineer(data)

shape after engineering (458913, 918)


In [9]:
data.head()

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,0,...,1,13,0,1,13,-1,1,13,6,1
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,6,...,1,13,0,1,13,-1,1,13,6,1
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,0,...,1,13,2,1,13,-1,1,13,6,1
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,0,...,1,13,0,1,13,-1,1,13,3,3
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.891679,0.042325,0.805045,0.940382,0.871900,0.000000,0.000000,0,0,0,...,1,13,0,1,13,1,1,13,6,1


In [10]:
data.isnull().sum()

P_2_mean        2434
P_2_std         7829
P_2_min         2434
P_2_max         2434
P_2_last        2434
                ... 
D_66_last          0
D_66_nunique       0
D_68_count         0
D_68_last          0
D_68_nunique       0
Length: 918, dtype: int64

As is our data has many null values, however we know from the data analysis done that a value being null conveys some meaning, so we will impute them with a value that is not present in the datset (-127)

In [11]:
data = data.fillna(-127)

In [12]:
# adding targets to dataset
data = pd.merge(data, targets, on='customer_ID')

In [13]:
# exporting datset
data.to_csv('amex_agg_data.csv', index=False)